In [1]:
import pandas as pd
import xlsxwriter
from os import walk
from def_zhanshi import *
from def_fiveTables import *
from def_fiveTables2 import *
from def_SummarySheets import *
from combin2csv import *
from datetime import datetime,timedelta
import time
# 忽略无用警告
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

d:\python\envs\jupyterpy36\scripts\python.exe


In [2]:
print("before set display.max_rows = ",pd.get_option("display.max_rows"))
pd.set_option("display.max_rows",80)
print("after set display.max_rows = ",pd.get_option("display.max_rows"))

before set display.max_rows =  8000
after set display.max_rows =  80


In [3]:
pd.set_option('display.large_repr', 'info')

In [4]:
import numpy as np
np.set_printoptions(suppress=True, threshold=np.nan)

In [5]:
date = '19/06/18'  # 表中日期
filename = 'testALL.xlsx'  # 输出文件名
inputPath = "G:/work/daily/DataShow/190618/"  # 输出路径
inputPath2 = "G:/work/daily/DataShow/"

In [6]:
date1 = '190618'  # 文件名日期
date2 = '2019-06-18'
date3 = '190617'

In [7]:
san_days  = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=3)).strftime('%Y-%m-%d')
qi_days   = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')
erwu_days = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=25)).strftime('%Y-%m-%d')
print("三日前：" + str(san_days))
print("七日前：" + str(qi_days))
print("二十五日前：" + str(erwu_days))

三日前：2019-06-15
七日前：2019-06-11
二十五日前：2019-05-24


In [8]:
df1 = pd.read_csv(inputPath + "data1.csv")
# df1["销售品编号"] = round(df1["销售品编号"],0)
# df1 = pd.read_excel(inputPath + "data1.xlsx")
split1 = pd.DataFrame((x.split(' ') for x in df1['订单生成时间']),index=df1.index,columns=['订单生成日期','订单生成小时'])
df1 = pd.merge(df1, split1, left_index=True, right_index=True)
# df1.to_csv(inputPath+"df1.csv",index=False)

In [9]:
df2 = pd.read_csv(inputPath + "NewPro.csv")
# df2 = pd.read_excel(inputPath + "NewPro.xlsx")
split2 = pd.DataFrame((x.split(' ') for x in df2['订单生成时间']),index=df2.index,columns=['订单生成日期','订单生成小时'])
df2 = pd.merge(df2, split2, left_index=True, right_index=True)


In [10]:
df3 = pd.read_csv(inputPath2 + "CHZH_Info"+ date3 + ".csv") # 首充历史表


In [11]:
df4 = pd.read_csv(inputPath + "NewPay" + date1 + ".csv") # 当日历史表
split4 = pd.DataFrame((x.split(' ') for x in df4['订单生成时间']), index=df4.index, columns=['订单生成日期', '订单生成小时'])
df4 = pd.merge(df4, split4, left_index=True, right_index=True)

In [12]:
df5 = pd.read_csv(inputPath + "NewJD.csv")
split5 = pd.DataFrame((x.split(' ') for x in df5['用户下单时间']),index=df5.index,columns=['用户下单日期','用户下单小时'])
df5 = pd.merge(df5, split5, left_index=True, right_index=True)


In [13]:
df1 = df1[df1["订单生成日期"] == san_days]
df2 = df2[df2["订单生成日期"] == san_days]
df5 = df5[df5["用户下单日期"] == san_days] # 京东表

In [14]:
print("df1:"+str(df1.shape))
print("df2:"+str(df2.shape))
print("df3:"+str(df3.shape))
print("df4:"+str(df4.shape))
print("df5:"+str(df5.shape))

df1:(17756, 97)
df2:(28355, 37)
df3:(61698, 97)
df4:(51703, 97)
df5:(11792, 38)


In [15]:
df1 = df1[(True ^ df1['订单状态'].isin(['办理成功', '办理中', '办理失败']))]
df1 = df1[pd.notnull(df1['所在省 / 市 / 县'])]
df1 = df1[df1["订单编号"].str.contains('订单编号') == False]
df1 = df1.drop_duplicates(subset=['订单编号'], keep='first')
df1 = df1[pd.notnull(df1['号码归属地'])]
df1 = df1[pd.notnull(df1['入网手机号'])]
df1["入网手机号"] = df1['入网手机号'].astype('int64')

In [16]:
df2 = df2[df2["模式分类"] == "京东模式"]
df2 = df2[pd.notnull(df2['是否下省'])]
df2.shape

(12445, 37)

In [17]:
df34 = pd.concat([df3, df4],sort=False)
df34 = df34.drop_duplicates(subset=['主号码'], keep='first') # 主号码
df34 = df34[pd.notnull(df34['主号码'])]
df34["主号码"] = df34['主号码'].astype('int64')
print("当前日期：" + date2)
ggDays = get_nday_list(date2, 7)
df34 = df34[df34["订单生成日期"].isin(ggDays)]


当前日期：2019-06-18
['2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-15', '2019-06-16', '2019-06-17']


In [18]:
df51 = df5[["运营商单号", "物流单号"]]
df52 = df5[["运营商单号", "APP操作时间"]]
df51 = df51[pd.notnull(df51['物流单号'])]
df52 = df52[pd.notnull(df52['APP操作时间'])]

In [19]:
print("df51:"+str(df51.shape))
print("df52:"+str(df52.shape))

df51:(11713, 2)
df52:(6015, 2)


In [20]:
df11 = pd.merge(df1, df2[["订单编号", "模式分类", "是否下省"]],
                left_on="订单编号", right_on="订单编号", how="left")
df11 = pd.merge(df11, df51, left_on="订单编号", right_on="运营商单号", how="left")
df11 = pd.merge(df11, df52, left_on="订单编号", right_on="运营商单号", how="left")
df11['入网手机号'] = df11['入网手机号'].apply(int)
df11 = pd.merge(df11, df34[['主号码']],left_on="入网手机号", right_on="主号码", how="left") #'订单编号',
# df11 = df11[df11["订单状态"] == "交易完成"]
df11["主号码_y"] = df11["主号码_y"].fillna(-1)
df11["主号码_y"] = df11['主号码_y'].astype('int64')

In [21]:
df12 = df11.copy()

In [22]:
def shouchong(x):
    if x == -1:
        return 0
    else:
        return 1

In [23]:
def laidan(x):
    if x is np.nan:
        return 0
    else:
        return 1


def fahuo(a,b,c):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    elif a is np.nan and c is not np.nan:
        return 1
    else:
        return 0

def qianshou(a,b):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    else:
        return 0

def jihuo(x):
    if x == "交易完成":
        return 1
    else:
        return 0

def paidan(x):
    if x is np.nan:
        return 0
    else:
        return 1

In [24]:
df11['派单'] = df11["是否下省"].apply(paidan)
df11['派卡'] = df11["物流单号_y"].apply(paidan)
df11['上门'] = df11["APP操作时间"].apply(paidan)
df11['首充'] = df11["主号码_y"].apply(shouchong) #订单编号_y
df11['来单量'] = df11['订单编号'].apply(laidan) #订单编号_x
df11['发货量'] = df11.apply(lambda x: fahuo(x["物流单号_x"], x["订单状态"], x["物流签收时间"]), axis=1)
df11['签收量'] = df11.apply(lambda x: qianshou(x["物流签收时间"], x["订单状态"]), axis=1)
df11['激活量'] = df11["订单状态"].apply(jihuo)

In [25]:
df = df11[["号码归属地","销售品编号","营业厅送货方式","派单","派卡","上门","首充","发货量","签收量","激活量","来单量"]]
split1 = pd.DataFrame((x.split('/') for x in df['号码归属地']),index=df.index,columns=['所属省','所属市'])
df = pd.merge(df, split1, left_index=True, right_index=True)

In [26]:
df["销售品编号"] = df["销售品编号"].map(lambda x : str(x))

In [27]:
# 产品标卡
biaoka = pd.read_excel(inputPath2 + "产品标卡.xlsx")
biaoka["销售品编号"] = biaoka["销售品编号"].map(lambda x: str(x))

In [28]:
biaoka["销售品编号"][0:5]

0    3001146112
1    3001147403
2    3001147680
3    3001156081
4    3001166277
Name: 销售品编号, dtype: object

In [29]:
df["销售品编号"] = df["销售品编号"].map(lambda x : x[:-2])

In [30]:
df["销售品编号"][0:5]

0    903011143780
1     90011235593
2     10011229051
3     10011228990
4     10011228964
Name: 销售品编号, dtype: object

In [31]:
data_JM3 = pd.merge(df, biaoka, how="left", on="销售品编号")


In [32]:
data_JM3["分类"]

0           新装
1           新装
2          京粉卡
3          京粉卡
4          抖音卡
5          京粉卡
6         阿里鱼卡
7         百度圣卡
8          抖音卡
9          V粉卡
10         V粉卡
11        百度圣卡
12         V粉卡
13         抖音卡
14        百度圣卡
15        阿里鱼卡
16         抖音卡
17         V粉卡
18         京粉卡
19       网易白金卡
20         抖音卡
21          新装
22          新装
23         抖音卡
24        百度圣卡
25        阿里鱼卡
26        百度圣卡
27        百度圣卡
28          新装
29         抖音卡
30         抖音卡
31          新装
32         抖音卡
33         抖音卡
34         V粉卡
35       网易白金卡
36       网易白金卡
37         V粉卡
38         京粉卡
39         京粉卡
         ...  
17497       新装
17498      V粉卡
17499      抖音卡
17500       新装
17501      京粉卡
17502      V粉卡
17503     百度圣卡
17504       新装
17505      京粉卡
17506      抖音卡
17507       新装
17508      京粉卡
17509      V粉卡
17510      抖音卡
17511      京粉卡
17512      京粉卡
17513      京粉卡
17514      抖音卡
17515      京粉卡
17516      京粉卡
17517    网易白金卡
17518      抖音卡
17519      京粉卡
17520      V粉卡
17521     百度圣卡
17522     

In [36]:
data10 = JD_mode_product(data_JM3)


In [37]:
data10.columns

Index(['分类', '派单', '产品派单量占比', '派卡', '派卡率', '异常筛查', '异常筛查率', '上门', '上门率', '未上门',
       '未上门率', '激活量', '产品激活量占比', '激活率', '首充', '产品首充量占比', '首充率', '全量首充'],
      dtype='object')

In [38]:
data10["分类"]

0       V粉卡
1       京粉卡
2       大王卡
3      大黑牛卡
4       抖音卡
5      百度圣卡
6     米粉卡体验
7     米粉卡日租
8     网易白金卡
9      阿里鱼卡
10       合计
Name: 分类, dtype: object